In [102]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/alfa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [103]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
import json
import re
from numpy import linalg as la

In [104]:
docs_path = 'data/tw_hurricane_data.json'
tweets_title = 'data/tweet_document_ids_map.csv'

tweets_id_title = {}

with open(tweets_title) as fp:
    lines = fp.readlines()


for l in lines:
    l = l.strip().split("\t")
    tweets_id_title[int(l[1])] =  l[0]


tweets = []
lines = []

for line in open(docs_path, 'r'):
    lines.append(line)
    #media = json.loads(line).get('entities').get('media')
    tweets.append({
        'id' : int(json.loads(line).get('id')),
        'text': json.loads(line).get('full_text'),
        'username' : json.loads(line).get('user').get('name'),
        'date' : json.loads(line).get('created_at'),
        'hashtag' : json.loads(line).get('entities').get('hashtags'), # Si añadimos .get('text') obtenemos solo info del hashtag
        'like' : json.loads(line).get('favorite_count'),
        'rt' : json.loads(line).get('retweet_count'),
        'URL' : len(json.loads(line).get('entities').get('urls'))
    }) #Here we get only the text from the tweets in json document
print(tweets[500])

{'id': 1575911964555247616, 'text': '#HurricaneIan expected to make less severe landfall impact in South Carolina within next 1-2 hours. https://t.co/IS8aEbO90o', 'username': 'AviationEnthusiastsClub', 'date': 'Fri Sep 30 18:14:26 +0000 2022', 'hashtag': [{'text': 'HurricaneIan', 'indices': [0, 13]}], 'like': 0, 'rt': 0, 'URL': 1}


In [105]:
print("Total number of Tweets: {}".format(len(tweets)))

Total number of Tweets: 4000


In [106]:
#This function will clean our text from data that is not important so that has no weight 
def clean_text(tweet):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    ## START CODE
   
    tweet = tweet.lower() # Transform in lowercase
    tweet = re.sub(r'[^\w\s]', '', tweet) # Here we remove punctuation marks
    tweet = tweet.split() # Tokenize the text to get a list of terms
    tweet = [word for word in tweet if word not in stop_words]  # eliminate the stopwords
    #tweet = [stemmer.stem(tweet) for tweet in tweets] # Perform stemming 
    ## END CODE    
    return tweet

In [107]:
print(clean_text(tweets[1000].get('text')))

['list', 'expanded', 'community', 'foundations', 'across', 'florida', 'set', 'relief', 'funds', 'help', 'impacted', 'hurricaneian', 'take', 'action', 'share', 'cfsarasota', 'manateecf', 'colliercffl', 'givecf', 'miamifoundation', 'httpstco0uui16fita']


In [108]:
def create_index(tweets):
    """
    Implement the inverted index
    
    Argument:
    lines -- collection of Wikipedia articles
    
    Returns:
    index - the inverted index (implemented through a Python dictionary) containing terms as keys and the corresponding
    list of documents where these keys appears in (and the positions) as values.
    """
    index = defaultdict(list)    

    for i in range(len(tweets)):
        terms = clean_text(tweets[i].get("text"))
        tweet_id = tweets[i].get('id')

        tweet_title = tweets_id_title[tweet_id]

        if tweet_title != None:
            tweet_title = tweet_title.get("title")

        current_tweet_index = {}

        for position, term in enumerate(terms):
            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list

                ## START CODE
                current_tweet_index[term][1].append(position)  
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_tweet_index[term] = [tweet_title, array('I', [position])] #'I' indicates unsigned int (int in Python)

        #merge the current page index with the main index
        for term, posting_page in current_tweet_index.items():
            index[term].append(posting_page)

        ## END CODE                    

    return index

In [109]:
index = create_index(tweets)

print("Index results for the term 'researcher': {}\n".format(index['house']))
print("First 10 Index results for the term 'research': \n{}".format(index['house'][:10]))

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/alfa/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_20589/2791291513.py", line 1, in <module>
    index = create_index(tweets)
  File "/tmp/ipykernel_20589/3619136245.py", line 18, in create_index
    tweet_title = next((item for item in tweets_id_title if item["id"] == tweet_id), None)
  File "/tmp/ipykernel_20589/3619136245.py", line 18, in <genexpr>
    tweet_title = next((item for item in tweets_id_title if item["id"] == tweet_id), None)
TypeError: 'int' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/alfa/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1997, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/alfa/.local/lib/python3.8/site-packages/IPython/co

In [110]:
def search(query, index):
    """
    The output is the list of documents that contain any of the query terms. 
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = clean_text(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"
            term_docs = [posting[0] for posting in index[term]]
            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    return docs

In [111]:
print("Insert your query (i.e.: Computer Science):\n")
query = input()
docs = search(query, index)

top = 10

print("\n======================\nSample of {} results out of {} for the searched query:\n".format(top, len(docs)))

for d_id in docs[:top]:
    print("page_id= {} - page_title: {}".format(d_id, tweets_id_title[d_id]))

Insert your query (i.e.: Computer Science):


Sample of 10 results out of 0 for the searched query:

